In [2]:
pip install numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# ЗАДАЧА 1
import numpy as np
import pandas as pd
# Вход векторы
x_list = [
    np.array([0.2, -0.4, 0.1]),
    np.array([1.0, 0.5, -0.3]),
    np.array([-0.1, -0.2, 0.7]),
    np.array([0.0, 0.0, 0.0]),
    np.array([2.0, -1.0, 0.3]),
]

# Парамеры юнита
w = np.array([0.5, -1.2, 0.8])
b = 0.4

# Активация
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

# Нейронный юнит
def unit(x, w, b, activation):
# y = activation(w·x + b)"
    z = np.dot(w, x) + b
    return activation(z)

# Вычисление выходов для всех входов и всех активаций
results = []

activations = {
    "sigmoid": sigmoid,
    "tanh": tanh,
    "relu": relu,
}

for i, x in enumerate(x_list, start=1):
    row = {"input": f"x{i}"}
    for name, act in activations.items():
        row[name] = unit(x, w, b, act)
    results.append(row)

df = pd.DataFrame(results)
print(df.round(4))

  input  sigmoid    tanh  relu
0    x1   0.7427  0.7857  1.06
1    x2   0.5150  0.0599  0.06
2    x3   0.7595  0.8178  1.15
3    x4   0.5987  0.3799  0.40
4    x5   0.9448  0.9932  2.84


In [8]:
# ЗАДАЧА 2

# XOR
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
], dtype=float)

y = np.array([[0], [1], [1], [0]], dtype=float)

np.random.seed(42)

# Веса
W1 = np.random.randn(2, 2) * 0.1
b1 = np.zeros((1, 2))
W2 = np.random.randn(2, 1) * 0.1
b2 = np.zeros((1, 1))

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def binary_cross_entropy(y_pred, y_true):
    eps = 1e-8
    return -np.mean(y_true * np.log(y_pred + eps) + (1 - y_true) * np.log(1 - y_pred + eps))

lr = 0.5 # Шаг
n_epochs = 10000

losses = []

for epoch in range(n_epochs):
    # Forward
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    y_pred = sigmoid(z2)
    
    # Потери
    loss = binary_cross_entropy(y_pred, y)
    losses.append(loss)
    
    # Backward
    grad_z2 = y_pred - y
    grad_W2 = np.dot(a1.T, grad_z2)
    grad_b2 = np.sum(grad_z2, axis=0, keepdims=True)
    
    # Градиентный спуск
    grad_a1 = np.dot(grad_z2, W2.T)
    grad_z1 = grad_a1 * relu_deriv(z1)
    grad_W1 = np.dot(X.T, grad_z1)
    grad_b1 = np.sum(grad_z1, axis=0, keepdims=True)
    
    # Обновление параметров
    W2 -= lr * grad_W2
    b2 -= lr * grad_b2
    W1 -= lr * grad_W1
    b1 -= lr * grad_b1

    if epoch % 2000 == 0:
        print(f"Эпоха {epoch}, Потери {loss:.6f}")

z1 = np.dot(X, W1) + b1
a1 = relu(z1)
z2 = np.dot(a1, W2) + b2
y_pred_final = sigmoid(z2)

for i, (input_val, pred, true) in enumerate(zip(X, y_pred_final, y)):
    print(f"{input_val}, Pred: {pred[0]:.4f}, True: {true[0]}, class: {int(pred[0] > 0.5)}")

accuracy = np.mean((y_pred_final > 0.5).astype(int) == y)
print(f"Точность: {accuracy:.4f}")

Эпоха 0, Потери 0.693190
Эпоха 2000, Потери 0.477393
Эпоха 4000, Потери 0.477665
Эпоха 6000, Потери 0.477390
Эпоха 8000, Потери 0.477388
[0. 0.], Pred: 0.6667, True: 0.0, class: 1
[0. 1.], Pred: 0.6667, True: 1.0, class: 1
[1. 0.], Pred: 0.6667, True: 1.0, class: 1
[1. 1.], Pred: 0.0000, True: 0.0, class: 0
Точность: 0.7500


In [9]:
# ЗАДАЧА 3
vocab = ["great", "excellent", "good", "bad", "awful", "movie"]
word2idx = {w: i for i, w in enumerate(vocab)}
d = 8  # размерность эмбеддинга

np.random.seed(42)
E = np.random.randn(len(vocab), d) * 0.1 # матрица эмбеддингов

train_data = [
    ("great movie", 1),
    ("excellent movie", 1),
    ("good movie", 1),
    ("bad movie", 0),
    ("awful movie", 0),
]

test_data = [
    ("great", 1),
    ("awful", 0),
    ("good movie", 1),
]

def tokenize(text):
    return text.lower().split()

def phrase_embedding(text):
    tokens = tokenize(text)
    vecs = []
    for t in tokens:
        if t in word2idx:
            vecs.append(E[word2idx[t]])
    if not vecs:
        return np.zeros(d)
    vecs = np.stack(vecs, axis=0)
    return vecs.mean(axis=0)

# Построим обучающие матрицы X, y
X_train = np.stack([phrase_embedding(text) for text, label in train_data], axis=0)
y_train = np.array([label for text, label in train_data])

X_test = np.stack([phrase_embedding(text) for text, label in test_data], axis=0)
y_test = np.array([label for text, label in test_data])

# Параметры сети 8 -> 4 -> 2
hidden_dim = 4
output_dim = 2

W1 = np.random.randn(d, hidden_dim) * 0.1
b1 = np.zeros((1, hidden_dim))
W2 = np.random.randn(hidden_dim, output_dim) * 0.1
b2 = np.zeros((1, output_dim))

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    x_shift = x - np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x_shift)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    num_classes = y_pred.shape[1]
    y_one_hot = np.eye(num_classes)[y_true]
    eps = 1e-8
    return -np.mean(np.sum(y_one_hot * np.log(y_pred + eps), axis=1))

lr = 0.1
n_epochs = 2000

for epoch in range(n_epochs):
    # Forward
    z1 = np.dot(X_train, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    y_pred = softmax(z2)
    
    # Loss
    loss = cross_entropy_loss(y_pred, y_train)
    
    # Backward
    grad_z2 = y_pred - np.eye(output_dim)[y_train]
    grad_z2 /= len(y_train)
    
    grad_W2 = np.dot(a1.T, grad_z2)
    grad_b2 = np.sum(grad_z2, axis=0, keepdims=True)
    
    grad_a1 = np.dot(grad_z2, W2.T)
    grad_z1 = grad_a1 * (a1 > 0)
    grad_W1 = np.dot(X_train.T, grad_z1)
    grad_b1 = np.sum(grad_z1, axis=0, keepdims=True)
    
    # Обновление
    W2 -= lr * grad_W2
    b2 -= lr * grad_b2
    W1 -= lr * grad_W1
    b1 -= lr * grad_b1
    
    if epoch % 400 == 0:
        print(f"Эпоха {epoch}, Потеря {loss:.4f}")

z1_test = np.dot(X_test, W1) + b1
a1_test = relu(z1_test)
z2_test = np.dot(a1_test, W2) + b2
y_pred_test = softmax(z2_test)
predictions = np.argmax(y_pred_test, axis=1)

accuracy = np.mean(predictions == y_test)
for i, (text, true_label) in enumerate(test_data):
    pred_prob = y_pred_test[i][1]
    pred_class = predictions[i]
    print(f"'{text}', True: {true_label}, Pred: {pred_class}, Prob: {pred_prob:.4f}")
print()

Эпоха 0, Потеря 0.6933
Эпоха 400, Потеря 0.6728
Эпоха 800, Потеря 0.6708
Эпоха 1200, Потеря 0.6498
Эпоха 1600, Потеря 0.4795
'great', True: 1, Pred: 1, Prob: 0.9875
'awful', True: 0, Pred: 0, Prob: 0.2410
'good movie', True: 1, Pred: 1, Prob: 0.9712



In [10]:
# ЗАДАЧА 4

corpus = [
    "i like neural networks",
    "i like deep learning",
    "neural networks are powerful",
    "deep learning is useful",
    "i like useful tools",
    "networks are useful",
]

# Построим слвоарь
tokens = set()
for sent in corpus:
    tokens.update(sent.split())

vocab = sorted(tokens)
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

vocab_size = len(vocab)
d = 6
np.random.seed(42)
E = np.random.randn(vocab_size, d) * 0.1 # Эмбеддинги

N = 3  # размер окна контекста

def make_dataset(corpus, N):
    X = []
    y = []
    for sent in corpus:
        words = sent.split()
        if len(words) <= N:
            continue
        idxs = [word2idx[w] for w in words]
        for t in range(N, len(idxs)):
            context = idxs[t-N:t] # N предыдущих слов
            target = idxs[t] # следущее слово
            X.append(context)
            y.append(target)
    return np.array(X), np.array(y)

X_ctx, y_tgt = make_dataset(corpus, N)

def get_embedding_for_context(context_ids):
    vecs = [E[i] for i in context_ids]
    return np.concatenate(vecs, axis=0)

# Построим матрицу признаков
X_embed = np.stack([get_embedding_for_context(ctx) for ctx in X_ctx], axis=0)

input_dim = N * d
hidden_dim = 16
output_dim = vocab_size

W = np.random.randn(input_dim, hidden_dim) * 0.1
b = np.zeros((1, hidden_dim))
U = np.random.randn(hidden_dim, output_dim) * 0.1
c = np.zeros((1, output_dim))

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    x_shift = x - np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x_shift)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    y_one_hot = np.eye(output_dim)[y_true]
    eps = 1e-8
    return -np.mean(np.sum(y_one_hot * np.log(y_pred + eps), axis=1))

def perplexity(loss):
    return np.exp(loss)

lr = 0.05
n_epochs = 2000

for epoch in range(n_epochs):
    # Forward
    z1 = np.dot(X_embed, W) + b
    h = relu(z1)
    z2 = np.dot(h, U) + c
    y_pred = softmax(z2)
    
    # Loss
    loss = cross_entropy_loss(y_pred, y_tgt)
    
    # Backward
    grad_z2 = y_pred - np.eye(output_dim)[y_tgt]
    grad_z2 /= len(y_tgt)
    
    grad_U = np.dot(h.T, grad_z2)
    grad_c = np.sum(grad_z2, axis=0, keepdims=True)
    
    grad_h = np.dot(grad_z2, U.T)
    grad_z1 = grad_h * (h > 0)
    grad_W = np.dot(X_embed.T, grad_z1)
    grad_b = np.sum(grad_z1, axis=0, keepdims=True)
    
    # Обновление
    W -= lr * grad_W
    b -= lr * grad_b
    U -= lr * grad_U
    c -= lr * grad_c
    
    if epoch % 400 == 0:
        ppl = perplexity(loss)
        print(f"Эпоха {epoch}, Потеря {loss:.4f}, Perplexity: {ppl:.4f}")

z1_final = np.dot(X_embed, W) + b
h_final = relu(z1_final)
z2_final = np.dot(h_final, U) + c
y_pred_final = softmax(z2_final)
final_loss = cross_entropy_loss(y_pred_final, y_tgt)
final_ppl = perplexity(final_loss)

print(f"Финальная потеря {final_loss:.4f}")
print(f"Финальная Perplexity: {final_ppl:.4f}")
for i in range(min(5, len(X_ctx))):
    context_words = [idx2word[idx] for idx in X_ctx[i]]
    target_word = idx2word[y_tgt[i]]
    pred_word_idx = np.argmax(y_pred_final[i])
    pred_word = idx2word[pred_word_idx]
    prob = y_pred_final[i][pred_word_idx]
    print(f"{context_words} -> True: '{target_word}', Pred: '{pred_word}' (p={prob:.4f})")
print()

Эпоха 0, Потеря 2.3980, Perplexity: 11.0012
Эпоха 400, Потеря 1.3414, Perplexity: 3.8244
Эпоха 800, Потеря 0.6462, Perplexity: 1.9083
Эпоха 1200, Потеря 0.3805, Perplexity: 1.4631
Эпоха 1600, Потеря 0.2693, Perplexity: 1.3091
Финальная потеря 0.1966
Финальная Perplexity: 1.2173
['i', 'like', 'neural'] -> True: 'networks', Pred: 'networks' (p=0.6236)
['i', 'like', 'deep'] -> True: 'learning', Pred: 'learning' (p=0.9102)
['neural', 'networks', 'are'] -> True: 'powerful', Pred: 'powerful' (p=0.9668)
['deep', 'learning', 'is'] -> True: 'useful', Pred: 'useful' (p=0.9836)
['i', 'like', 'useful'] -> True: 'tools', Pred: 'tools' (p=0.6932)



In [11]:
# ЗАДАЧА 5

np.random.seed(42)

# Вход и целевая метка
x = np.array([[0.3, -0.1, 0.0, 1.0, 2.0]])
y = np.array([1])

input_dim = 5
hidden_dim = 4
output_dim = 3

# Инициализация параметров
def init_params():
    W1 = np.random.uniform(-0.1, 0.1, size=(input_dim, hidden_dim))
    b1 = np.random.uniform(-0.1, 0.1, size=(1, hidden_dim))
    W2 = np.random.uniform(-0.1, 0.1, size=(hidden_dim, output_dim))
    b2 = np.random.uniform(-0.1, 0.1, size=(1, output_dim))
    return W1, b1, W2, b2

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    x_shift = x - np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x_shift)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    y_one_hot = np.eye(output_dim)[y_true]
    eps = 1e-8
    return -np.mean(np.sum(y_one_hot * np.log(y_pred + eps), axis=1))

def forward(x, W1, b1, W2, b2):
    z1 = np.dot(x, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    y_pred = softmax(z2)
    cache = {'z1': z1, 'a1': a1, 'z2': z2, 'y_pred': y_pred}
    return y_pred, cache

def backward(x, y, W1, b1, W2, b2, cache):
    z1, a1, z2, y_pred = cache['z1'], cache['a1'], cache['z2'], cache['y_pred']
    
    # Градиент по выходному слою
    grad_z2 = y_pred - np.eye(output_dim)[y]
    grad_z2 /= len(y)
    
    grad_W2 = np.dot(a1.T, grad_z2)
    grad_b2 = np.sum(grad_z2, axis=0, keepdims=True)
    
    # Градиент по скрытому слою
    grad_a1 = np.dot(grad_z2, W2.T)
    grad_z1 = grad_a1 * relu_deriv(z1)
    grad_W1 = np.dot(x.T, grad_z1)
    grad_b1 = np.sum(grad_z1, axis=0, keepdims=True)
    
    return grad_W1, grad_b1, grad_W2, grad_b2

def numerical_gradient(param, f, eps=1e-5):
    grad = np.zeros_like(param)
    it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        original_value = param[idx]

        param[idx] = original_value + eps
        loss_plus = f(param)

        param[idx] = original_value - eps
        loss_minus = f(param)

        grad[idx] = (loss_plus - loss_minus) / (2 * eps)
        param[idx] = original_value
        it.iternext()
    return grad

# Инициализация и вычисление аналитических градиентов
W1, b1, W2, b2 = init_params()
y_pred, cache = forward(x, W1, b1, W2, b2)
loss = cross_entropy_loss(y_pred, y)
grad_W1_analytical, grad_b1_analytical, grad_W2_analytical, grad_b2_analytical = backward(x, y, W1, b1, W2, b2, cache)

# Проверка градиентов численным методом
def f_W1(W1_perturbed):
    y_pred, _ = forward(x, W1_perturbed, b1, W2, b2)
    return cross_entropy_loss(y_pred, y)

def f_b1(b1_perturbed):
    y_pred, _ = forward(x, W1, b1_perturbed, W2, b2)
    return cross_entropy_loss(y_pred, y)

def f_W2(W2_perturbed):
    y_pred, _ = forward(x, W1, b1, W2_perturbed, b2)
    return cross_entropy_loss(y_pred, y)

def f_b2(b2_perturbed):
    y_pred, _ = forward(x, W1, b1, W2, b2_perturbed)
    return cross_entropy_loss(y_pred, y)

grad_W1_numerical = numerical_gradient(W1.copy(), f_W1)
grad_b1_numerical = numerical_gradient(b1.copy(), f_b1)
grad_W2_numerical = numerical_gradient(W2.copy(), f_W2)
grad_b2_numerical = numerical_gradient(b2.copy(), f_b2)

# Сравнение
max_diff_W1 = np.max(np.abs(grad_W1_analytical - grad_W1_numerical))
max_diff_b1 = np.max(np.abs(grad_b1_analytical - grad_b1_numerical))
max_diff_W2 = np.max(np.abs(grad_W2_analytical - grad_W2_numerical))
max_diff_b2 = np.max(np.abs(grad_b2_analytical - grad_b2_numerical))

max_diff_overall = max(max_diff_W1, max_diff_b1, max_diff_W2, max_diff_b2)

print("Задача 5 - Проверка градиентов:")
print(f"Максимальная разница для W1: {max_diff_W1:.8f}")
print(f"Максимальная разница для b1: {max_diff_b1:.8f}")
print(f"Максимальная разница для W2: {max_diff_W2:.8f}")
print(f"Максимальная разница для b2: {max_diff_b2:.8f}")
print(f"Общая максимальная разница: {max_diff_overall:.8f}")

Задача 5 - Проверка градиентов:
Максимальная разница для W1: 0.00000000
Максимальная разница для b1: 0.00000000
Максимальная разница для W2: 0.00000000
Максимальная разница для b2: 0.00000002
Общая максимальная разница: 0.00000002
